In [1]:
!pip uninstall transformers accelerate -y
!pip install transformers[torch]

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 860.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/1

In [2]:
!pip install accelerate -U

In [7]:
!pip install kaggle

In [6]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
!wget https://drive.google.com/file/d/1YOZVY6uDW6DjIIQ55AE48RXbpyfTWuXR/view?usp=drive_link

--2024-04-06 12:03:09--  https://drive.google.com/file/d/1YOZVY6uDW6DjIIQ55AE48RXbpyfTWuXR/view?usp=drive_link
Resolving drive.google.com (drive.google.com)... 142.251.16.101, 142.251.16.139, 142.251.16.100, ...
Connecting to drive.google.com (drive.google.com)|142.251.16.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=drive_link’

view?usp=drive_link     [ <=>                ]  83.48K  --.-KB/s    in 0.007s  

2024-04-06 12:03:09 (12.3 MB/s) - ‘view?usp=drive_link’ saved [85480]



In [4]:
!cp kaggle.json ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [8]:
!unzip /content/amazon-fine-food-reviews.zip

Archive:  /content/amazon-fine-food-reviews.zip
  inflating: Reviews.csv             
  inflating: database.sqlite         
  inflating: hashes.txt              


In [10]:
!unzip /content/app.zip

Archive:  /content/app.zip
  inflating: app.py                  
  inflating: book.html               
  inflating: index.html              
  inflating: menu.html               
  inflating: review.html             
replace reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: reviews.csv             
  inflating: static/css/bootstrap.css  
  inflating: static/css/font-awesome.min.css  
  inflating: static/css/responsive.css  
  inflating: static/css/style.css    
  inflating: static/css/style.css.map  
  inflating: static/css/style.scss   
  inflating: static/fonts/fontawesome-webfont.ttf  
  inflating: static/fonts/fontawesome-webfont.woff  
  inflating: static/fonts/fontawesome-webfont.woff2  
  inflating: static/images/about-img.png  
  inflating: static/images/client1.jpg  
  inflating: static/images/client2.jpg  
  inflating: static/images/f1.png    
  inflating: static/images/f2.png    
  inflating: static/images/f3.png    
  inflating: static/images/f4.png    
  in

In [13]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

dataset_path = 'Reviews.csv'
df = pd.read_csv(dataset_path, nrows=500)

train_texts, val_texts, train_summaries, val_summaries = train_test_split(
    df['Text'].tolist(), df['Summary'].tolist(), test_size=0.1, random_state=42
)

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

train_encodings = tokenizer(train_texts, max_length=512, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, max_length=512, truncation=True, padding=True)

train_labels = tokenizer(train_summaries, max_length=150, truncation=True, padding=True)
val_labels = tokenizer(val_summaries, max_length=150, truncation=True, padding=True)

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])
        return item

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

def data_collator(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=250,
    evaluation_strategy="steps",
    eval_steps=250,
    save_total_limit=3,
    overwrite_output_dir=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

trainer.train()

model.save_pretrained("fine_tuned_summarizer_model")
tokenizer.save_pretrained("fine_tuned_summarizer_tokenizer")


RuntimeError: Failed to import transformers.models.t5.modeling_t5 because of the following error (look up to see its traceback):
Failed to import transformers.integrations.peft because of the following error (look up to see its traceback):
partially initialized module 'torch._dynamo' has no attribute 'external_utils' (most likely due to a circular import)

In [14]:
!cp -r /content/fine_tuned_summarizer_tokenizer /content/drive/MyDrive/Ideathan/

cp: cannot stat '/content/fine_tuned_summarizer_tokenizer': No such file or directory


In [11]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Ideathan/fine_tuned_summarizer_model")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/Ideathan/fine_tuned_summarizer_tokenizer")

reviews = [
    "The prize of the pizza i ordered was compratively cheap than I expected",
    "I ordered the burger with fries, and it was disappointing. The burger was overcooked, and the fries were soggy. Not worth the price.",
    "The sushi was incredible! Fresh fish and perfectly seasoned rice. Will be my go-to sushi spot from now on.",
    "The pasta was underwhelming. The sauce lacked flavor, and the noodles were overcooked. Won't be returning.",
    "The steak was cooked to perfection! Juicy and flavorful, just the way I like it."
]
review_summaries = []
for review in reviews:
    inputs = tokenizer.encode("summarize: " + review, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=50, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    review_summaries.append(summary)

overall_summary = " ".join(review_summaries)

inputs = tokenizer.encode("summarize: " + overall_summary, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
final_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Overall Summary of Reviews:")
lst=list(final_summary.split('.'))
for i in lst:
    print(i)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Overall Summary of Reviews:
the pizza i ordered was compratively cheap than I expected
 the burger was overcooked, and the fries were soggy
 not worth the price
 the sauce lacked flavor, and the noodles were overcooked



In [ ]:
!unzip app.zip

Archive:  app.zip
  inflating: app.py                  
  inflating: book.html               
  inflating: index.html              
  inflating: menu.html               
  inflating: review.html             
  inflating: reviews.csv             
  inflating: static/css/bootstrap.css  
  inflating: static/css/font-awesome.min.css  
  inflating: static/css/responsive.css  
  inflating: static/css/style.css    
  inflating: static/css/style.css.map  
  inflating: static/css/style.scss   
  inflating: static/fonts/fontawesome-webfont.ttf  
  inflating: static/fonts/fontawesome-webfont.woff  
  inflating: static/fonts/fontawesome-webfont.woff2  
  inflating: static/images/about-img.png  
  inflating: static/images/client1.jpg  
  inflating: static/images/client2.jpg  
  inflating: static/images/f1.png    
  inflating: static/images/f2.png    
  inflating: static/images/f3.png    
  inflating: static/images/f4.png    
  inflating: static/images/f5.png    
  inflating: static/images/f6.png   

In [2]:
!pip install pyngrok

In [13]:
!ngrok authtoken 2b7Ilnk9Ki0v9Rm63sKaZzJrNzl_6rKfcUhpchy3LbtwZqhyH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, render_template
from transformers import T5ForConditionalGeneration, T5Tokenizer
import csv
from pyngrok import ngrok
import threading

ngrok.kill()
def start_ngrok():
    ngrok_address = ngrok.connect(5000)
    print(' * Tunnel URL:', ngrok_address)

threading.Thread(target=start_ngrok).start()
app = Flask(__name__)
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Ideathan/fine_tuned_summarizer_model")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/Ideathan/fine_tuned_summarizer_tokenizer")
def add_review_to_csv(food_item, review):
    with open('reviews.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([food_item, review])

@app.route('/')
def index():
    return render_template('review.html')

@app.route('/submit_review', methods=['POST'])
def submit_review():
    food_item = request.form['food_item']
    review = request.form['review']
    add_review_to_csv(food_item, review)
    return render_template('review.html')

@app.route('/summarize', methods=['POST'])
def summarize():
    reviews = []
    with open('reviews.csv', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            reviews.append(row['REVIEW'])
    review_summaries = []
    for review in reviews:
        inputs = tokenizer.encode("summarize: " + review, return_tensors="pt", max_length=50, truncation=True)
        outputs = model.generate(inputs, max_length=50, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        review_summaries.append(summary)
    overall_summary = " ".join(review_summaries)
    inputs = tokenizer.encode("summarize: " + overall_summary, return_tensors="pt", max_length=50, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    final_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return final_summary
if __name__ == '__main__':
    app.run()


 * Tunnel URL: NgrokTunnel: "https://91ca-34-86-62-198.ngrok-free.app" -> "http://localhost:5000"


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Apr/2024 12:15:40] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2024 12:15:40] "GET /static/css/bootstrap.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2024 12:15:40] "GET /static/css/responsive.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2024 12:15:40] "GET /static/css/font-awesome.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2024 12:15:40] "GET /static/images/f1.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2024 12:15:40] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2024 12:15:42] "GET /static/images/favicon.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Apr/2024 12:15:53] "POST /summarize HTTP/1.1" 200 -
